In [16]:
import numpy as np
import pickle
import pandas as pd
import os
import cv2 as cv
from skimage.transform import resize
import matplotlib.pyplot as plt
from transformers import ViltProcessor, ViltForQuestionAnswering, AutoTokenizer, AutoConfig
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [17]:
model_path = "dandelin/vilt-b32-finetuned-vqa"

config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True)  
processor = ViltProcessor.from_pretrained(model_path)
model = ViltForQuestionAnswering.from_pretrained(model_path, config=config)

In [18]:
yes_is_more_dirty = ["Is there trash?", "Is there garbage?", "Is there waste?", "Is there junk?", "Is there residue?", "Is there rubbish?", "Is there a lot of trash?", "Is there a lot of garbage?", "Is there a lot of waste?", "Is there a lot of junk?", "Is there a lot of residue?", "Is there a lot of rubbish?", "Is there any type of trash?", "Is there any type of garbage?", "Is there any type of waste?", "Is there any type of junk?", "Is there any type of residue?", "Is there any type of rubbish?", "Are there any bottles on the ground?", "Are there any bags on the ground?", "Is there any plastic on the ground?", "Is there any paper on the ground?", "Are thre any leaves on the ground?", "Are there plants on the ground?", "Are there any branches on the ground?", "Is there any metal on the ground?", "Are there any packages on the ground?", "Is there any stuff on the ground?", "Are there any times on the ground?", "Are there any objects on the ground?", "Is there anything in the water?", "Are there small items on the ground?"]
yes_is_less_dirty = ["Is there a lot of water?", "Is there water?", "Is there grass?", "Is there sand?", "Are there buildings?", "Are there structures?", "Are there bricks?", "Are there walls?", "Is the ground empty?", "Is the ground sand?", "Are there stones?", "Is there concrete", "Is the ground clean?", "Are there shadows on the ground?", "Are there dark and light areas on the ground?", "Are there houses?", "Does the ground have different colors?", "Are there people?", "Are there clothes?", "Is there laundry?", "Are there animals?", "Are there birds?", "Are there trees?"]

In [19]:
import random

questions_yes_is_more_dirty = random.sample(yes_is_more_dirty, 10)

questions_yes_is_less_dirty = random.sample(yes_is_less_dirty, 10)
    

In [20]:
questions_yes_is_more_dirty
questions_yes_is_less_dirty

['Is there concrete',
 'Are there stones?',
 'Are there trees?',
 'Are there structures?',
 'Is there sand?',
 'Does the ground have different colors?',
 'Are there people?',
 'Is the ground clean?',
 'Is the ground empty?',
 'Are there clothes?']

In [21]:
csv_file = 'stefan.csv'
df = pd.read_csv(csv_file)
raw = df[[ 'image1', 'image2', 'image3', 'image4',
          'image5', 'stefan', 'loc_id2']]

your_path = 'D:/Projects/2022/GarbageDetection/vilt'
data_path = 'D:/Projects/2022/GarbageDetection/images'
folders = os.listdir(data_path)

all_images = []
y = []

predictions = []

""" no_garbage_deadline= 3.0
little_garbage_deadline= 3.5
needs_to_be_checked_deadline= 4.25
needs_cleaning_deadline= 6 """

for index, row in raw.iterrows():
    #print(len(all_images))
    print("################################")
    idx_filename = 0
    #all_images_column = []
    sum_all_images=0
    filenames = [row.image1, row.image2, row.image3,
                    row.image4, row.image5, row.stefan]
    locid=row.loc_id2
    #decide how many images to use
    k=100
    if(len(all_images) == k):
        break
    for filename in filenames:
        # finish loc iterations appending groundtruth
        if(idx_filename == 5):
            """  pred='None'
            #print(all_images_column)
            if(sum_all_images< no_garbage_deadline):
                pred='1-Very clean'
            elif(sum_all_images< little_garbage_deadline):
                pred='2-Clean'
            elif(sum_all_images< needs_to_be_checked_deadline):
                pred='3-Not clean/ Not dirty'
            elif(sum_all_images< needs_cleaning_deadline):
                pred='4-Dirty'
            else:
                pred='5-Very dirty' """
            print('Prediction:', sum_all_images)
            print('Groundtruth:', filename)
            print('loc:', locid)
            predictions.append(sum_all_images)
            #y.append(filename)
            continue
        # search for image using name in local repo
        for folder in folders:
            files = os.listdir(f'{data_path}/{folder}')
            if filename in files:
                idx_filename += 1
                img = cv.imread(f'{data_path}/{folder}/{filename}')
                print(f'{data_path}/{folder}/{filename}')
                """ imgplot = plt.imshow(img)
                plt.show() """ 
                sum=0
                k=0
                ########
                for qst in questions_yes_is_more_dirty:
                    VQ_encoding = processor(img, qst, return_tensors="pt")
                    outputs = model(**VQ_encoding, return_dict = True)
                    logits = outputs.logits
                    idx = logits.argmax(-1).item()
                    #print(qst, (model.config.id2label[idx]))
                    if(model.config.id2label[idx] == "yes"):
                        sum+=1
                    k+=1
                #########
                for qst in questions_yes_is_less_dirty:
                    VQ_encoding = processor(img, qst, return_tensors="pt")
                    outputs = model(**VQ_encoding, return_dict = True)
                    logits = outputs.logits
                    idx = logits.argmax(-1).item()
                    #print(qst, (model.config.id2label[idx]))
                    if(model.config.id2label[idx] == "no"):
                        sum+=1
                    k+=1
                value= sum/k
                print(value*10)
                #print(value)
                #all_images_column.append(value*2)
                sum_all_images += value*2
#pickle.dump(all_images, open("all_images.bin", "wb"))
#pickle.dump(y, open("y_images.bin", "wb"))
#print(len(all_images), len(y))

aux = pd.DataFrame({
    "pred": predictions,
})

aux = raw.join(aux)

################################
D:/Projects/2022/GarbageDetection/images/uuid777c4368-5dfb-4fb4-a1aa-484a7e1eb48c/1638525270249.jpg
7.0
D:/Projects/2022/GarbageDetection/images/uuid777c4368-5dfb-4fb4-a1aa-484a7e1eb48c/1638525283559.jpg
6.5
D:/Projects/2022/GarbageDetection/images/uuid777c4368-5dfb-4fb4-a1aa-484a7e1eb48c/1638525306900.jpg
7.0
D:/Projects/2022/GarbageDetection/images/uuid777c4368-5dfb-4fb4-a1aa-484a7e1eb48c/1638525317033.jpg
6.5
D:/Projects/2022/GarbageDetection/images/uuid777c4368-5dfb-4fb4-a1aa-484a7e1eb48c/1638525327292.jpg
6.0
Prediction: 6.6
Groundtruth: 4
loc: 11
################################
D:/Projects/2022/GarbageDetection/images/uuid55e9e9df-9f30-4229-8c28-f6472348f579/1642660749942.jpg
6.0
D:/Projects/2022/GarbageDetection/images/uuid55e9e9df-9f30-4229-8c28-f6472348f579/1642660773116.jpg
5.5
D:/Projects/2022/GarbageDetection/images/uuid55e9e9df-9f30-4229-8c28-f6472348f579/1642660792953.jpg
7.0
D:/Projects/2022/GarbageDetection/images/uuid55e9e9df-9f30-4229

,image1,image2,image3,image4,image5,stefan,loc_id2,pred
0,1638525270249.jpg,1638525283559.jpg,1638525306900.jpg,1638525317033.jpg,1638525327292.jpg,4,11,6.6
1,1642660749942.jpg,1642660773116.jpg,1642660792953.jpg,1642660804540.jpg,1642660831081.jpg,4,11,6.1
2,1646117300053.jpg,1646117309681.jpg,1646117321138.jpg,1646117334390.jpg,1646117369467.jpg,4,11,5.0
3,1638519175551.jpg,1638519185450.jpg,1638519196014.jpg,1638519206614.jpg,1638519215506.jpg,1,19,4.6
4,1642663350233.jpg,1642663379608.jpg,1642663391243.jpg,1642663402581.jpg,1642663418001.jpg,1,19,2.8
...,...,...,...,...,...,...,...,...
115,1643299090506.jpg,1643299174679.jpg,1643299195451.jpg,1643299218108.jpg,1643299230485.jpg,2,103072,5.8
116,1647933334561.jpg,1647933339297.jpg,1647933346438.jpg,1647933353124.jpg,1647933362908.jpg,2,103077,6.7
117,1643272687833.jpg,1643272696175.jpg,1643272706526.jpg,1643272711888.jpg,1643272717904.jpg,1,304024,4.8
118,1647413422436.jpg,1647413427428.jpg,1647413446275.jpg,1647413457644.jpg,1647413464486.jpg,1,302075,6.0


In [22]:
#calculate accuracy


#table with accuracy and questions

#media of accuracy for each question

In [24]:
print(raw)

                image1             image2             image3  \
0    1638525270249.jpg  1638525283559.jpg  1638525306900.jpg   
1    1642660749942.jpg  1642660773116.jpg  1642660792953.jpg   
2    1646117300053.jpg  1646117309681.jpg  1646117321138.jpg   
3    1638519175551.jpg  1638519185450.jpg  1638519196014.jpg   
4    1642663350233.jpg  1642663379608.jpg  1642663391243.jpg   
..                 ...                ...                ...   
115  1643299090506.jpg  1643299174679.jpg  1643299195451.jpg   
116  1647933334561.jpg  1647933339297.jpg  1647933346438.jpg   
117  1643272687833.jpg  1643272696175.jpg  1643272706526.jpg   
118  1647413422436.jpg  1647413427428.jpg  1647413446275.jpg   
119  1647340010902.jpg  1647340017172.jpg  1647340027581.jpg   

                image4             image5  stefan  loc_id2  
0    1638525317033.jpg  1638525327292.jpg       4       11  
1    1642660804540.jpg  1642660831081.jpg       4       11  
2    1646117334390.jpg  1646117369467.jpg       